In [1]:
import torch

## A Simple Function

In [2]:
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [3]:
# Can also create x = torch.arange(4.0, requires_grad=True)
x.requires_grad_(True)
# The gradient is None by default
print(x.grad)

None


In [4]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [5]:
"""
We can now take the gradient of y with respect to x by calling its backward
method. Next, we can access the gradient via x’s grad attribute.
"""

y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [6]:
x.grad == 4 * x

tensor([True, True, True, True])

In [7]:
"""
Note that PyTorch does not automatically reset the gradient buffer when we
record a new gradient. Instead, the new gradient is added to the
already-stored gradient. This behavior comes in handy when we want to
optimize the sum of multiple objective functions. To reset the gradient
buffer, we can call x.grad.zero().
"""

x.grad.zero_()  # Reset the gradient
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

## Backward for Non-Scalar Variables

In [8]:
x.grad.zero_()
y = x * x
y.backward(gradient=torch.ones(len(y)))  # Faster: y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

## Detaching Computation

In [9]:
x.grad.zero_()  # Reset the gradient of x to zero
y = x * x  # Compute y = x * x
u = y.detach()  # Detach y to create u which has no gradient history
z = u * x  # Compute z = u * x
z.sum().backward()  # Perform backpropagation to compute the gradient of z with respect to x
x.grad == u  # Check if the gradient of x is equal to u

tensor([True, True, True, True])

In [10]:
"""
Note that while this procedure detaches y’s ancestors from the graph leading
to z, the computational graph leading to y persists and thus we can
calculate the gradient of y with respect to x.
"""

x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## Gradients and Python Control Flow

In [11]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [12]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

In [13]:
# it is a linear function of a with piecewise defined scale

a.grad == d / a

tensor(True)